In [ ]:
!pip install rouge

In [ ]:
import os
import spacy
import numpy as np
import pandas as pd
from rouge import Rouge
from heapq import nlargest
from itertools import islice
from string import punctuation 
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
data_path = "/content/drive/MyDrive/LegSuM/Data/catest_processed.csv"
data = pd.read_csv(data_path)

In [ ]:
data[['clean_text', 'summary']].head()

,clean_text,summary
0,The people of the State of California do enact...,Existing property tax law establishes a vetera...
1,The people of the State of California do enact...,Existing law provides that the Board of Parole...
2,The people of the State of California do enact...,The Sales and Use Tax Law imposes a tax on ret...
3,The people of the State of California do enact...,"Existing law requires all moneys, except for f..."
4,The people of the State of California do enact...,Existing law defines a request regarding resus...


For Single Cae

In [ ]:
text = data['clean_text'][0]

In [ ]:
stopwords = list(STOP_WORDS)
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(text)

In [ ]:
tokens = [token.text for token in doc]
print(tokens)

['The', 'people', 'of', 'the', 'State', 'of', 'California', 'do', 'enact', 'as', 'follows', ':', 'SECTIONHEADER', 'The', 'Legislature', 'finds', 'and', 'declares', 'all', 'of', 'the', 'following', ':', '(', '1', ')', 'Since', '1899', 'congressionally', 'chartered', 'veterans', '’', 'organizations', 'have', 'provided', 'a', 'valuable', 'service', 'to', 'our', 'nation', '’s', 'returning', 'service', 'members', '.', 'These', 'organizations', 'help', 'preserve', 'the', 'memories', 'and', 'incidents', 'of', 'the', 'great', 'hostilities', 'fought', 'by', 'our', 'nation', ',', 'and', 'preserve', 'and', 'strengthen', 'comradeship', 'among', 'members', '.', 'These', 'veterans', '’', 'organizations', 'also', 'own', 'and', 'manage', 'various', 'properties', 'including', 'lodges', ',', 'posts', ',', 'and', 'fraternal', 'halls', '.', 'These', 'properties', 'act', 'as', 'a', 'safe', 'haven', 'where', 'veterans', 'of', 'all', 'ages', 'and', 'their', 'families', 'can', 'gather', 'together', 'to', 'fin

In [ ]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punctuation = punctuation + '\n'

In [ ]:
def GetWordFrequency(doc):
  
  word_frequnecy_ = {}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequnecy_.keys():
          word_frequnecy_[word.text] = 1
        else:
          word_frequnecy_[word.text] += 1

  return word_frequnecy_

In [ ]:
word_frequnecy = GetWordFrequency(doc)

In [ ]:
max_frequency = max(word_frequnecy.values())

In [ ]:
for word in word_frequnecy.keys():
  word_frequnecy[word] = word_frequnecy[word] / max_frequency

In [ ]:
print(word_frequnecy)

{'people': 0.13043478260869565, 'State': 0.2608695652173913, 'California': 0.17391304347826086, 'enact': 0.043478260869565216, 'follows': 0.043478260869565216, 'SECTIONHEADER': 0.043478260869565216, 'Legislature': 0.08695652173913043, 'finds': 0.08695652173913043, 'declares': 0.08695652173913043, 'following': 0.08695652173913043, '1': 0.13043478260869565, '1899': 0.043478260869565216, 'congressionally': 0.043478260869565216, 'chartered': 0.13043478260869565, 'veterans': 1.0, '’': 0.6086956521739131, 'organizations': 0.5217391304347826, 'provided': 0.17391304347826086, 'valuable': 0.043478260869565216, 'service': 0.21739130434782608, 'nation': 0.08695652173913043, 'returning': 0.08695652173913043, 'members': 0.34782608695652173, 'help': 0.043478260869565216, 'preserve': 0.08695652173913043, 'memories': 0.043478260869565216, 'incidents': 0.043478260869565216, 'great': 0.043478260869565216, 'hostilities': 0.043478260869565216, 'fought': 0.043478260869565216, 'strengthen': 0.04347826086956

In [ ]:
sentence_tokens = [i for i in doc.sents]
sentence_tokens[:5]

[The people of the State of California do enact as follows: SECTIONHEADER,
 The Legislature finds and declares all of the following: (1),
 Since 1899 congressionally chartered veterans’ organizations have provided a valuable service to our nation’s returning service members.,
 These organizations help preserve the memories and incidents of the great hostilities fought by our nation, and preserve and strengthen comradeship among members.,
 These veterans’ organizations also own and manage various properties including lodges, posts, and fraternal halls.]

In [ ]:
def GetSentenceScore(sentence_tokens):

  sentence_score_ = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequnecy.keys():
        if sent not in sentence_score_.keys():
          sentence_score_[sent] = word_frequnecy[word.text.lower()]
        else:
          sentence_score_[sent] += word_frequnecy[word.text.lower()]
    
  return sentence_score_

In [ ]:
sentence_score = GetSentenceScore(sentence_tokens)

In [ ]:
def take(n, iterable):
    return list(islice(iterable, n))

In [ ]:
take(5, sentence_score.items())

[(The people of the State of California do enact as follows: SECTIONHEADER,
  0.2608695652173913),
 (The Legislature finds and declares all of the following: (1),
  0.3913043478260869),
 (Since 1899 congressionally chartered veterans’ organizations have provided a valuable service to our nation’s returning service members.,
  3.521739130434783),
 (These organizations help preserve the memories and incidents of the great hostilities fought by our nation, and preserve and strengthen comradeship among members.,
  1.478260869565217),
 (These veterans’ organizations also own and manage various properties including lodges, posts, and fraternal halls.,
  2.9565217391304355)]

In [ ]:
sentence_length = int(len(sentence_tokens)*0.1)
print(f"There are {sentence_length} which counts for 10% of Sentence with maximum socre. ")

There are 3 which counts for 10% of Sentence with maximum socre. 


In [ ]:
summary_list  = nlargest(sentence_length, sentence_score, key=sentence_score.get)

In [ ]:
Summary = ' '.join([word.text for word in summary_list])

In [ ]:
GoldSummary = data['summary'][0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(Summary, data['summary'][0])

In [ ]:
pd.DataFrame(score[0])

,rouge-1,rouge-2,rouge-l
r,0.474227,0.188312,0.381443
p,0.353846,0.133641,0.284615
f,0.405286,0.156334,0.325991


**For Whole Set**

In [ ]:
text = data['clean_text']
GoldSummaries =  data['summary']
SysSummaries = []

In [ ]:
def summaries_case(case):

  doc = nlp(case)
  tokens = [token.text for token in doc]
  word_frequnecy = GetWordFrequency(doc)
  max_frequency = max(word_frequnecy.values())

  for word in word_frequnecy.keys():
      word_frequnecy[word] = word_frequnecy[word] / max_frequency

  sentence_tokens = [i for i in doc.sents]
  sentence_score = GetSentenceScore(sentence_tokens)
  sentence_length = int(len(sentence_tokens)*0.1)

  summary_list  = nlargest(sentence_length, sentence_score, key=sentence_score.get)
  Summary = ' '.join([word.text for word in summary_list])

  return Summary

In [ ]:
for i, case in enumerate(data['clean_text']):

  try:
    summary = summaries_case(case)
    SysSummaries.append(summary)
    print(i)
  except Exception as e:
    SysSummaries.append(np.NaN)
    print(e, 'for' ,i)

In [ ]:
len(SysSummaries), len(GoldSummaries)

(1237, 1237)

In [ ]:
DFSUMResuts = pd.DataFrame(zip(SysSummaries, GoldSummaries), columns = ['SysSummaries', 'GoldSummaries'])

In [ ]:
TfIdfRouge = rouge.get_scores(DFSUMResuts['SysSummaries'], DFSUMResuts['GoldSummaries'], avg=True)

In [ ]:
TfIdfRouge = pd.DataFrame(TfIdfRouge).set_index([["recall", "precision", "f-measure"]])
TfIdfRouge

,rouge-1,rouge-2,rouge-l
recall,0.331921,0.152646,0.291707
precision,0.393599,0.200556,0.348015
f-measure,0.339730,0.159899,0.299285


In [ ]:
path = "/content/drive/MyDrive/LegSuM/scores/"

TfIdfRouge.to_csv(path + 'TfIdfRouge.csv', index=True, header=True)